In [1]:
#imports
import pickle
import torch
from torch import nn
import torchvision
import torch.optim as optim

In [2]:
#project specific imports
from modules.utils import *
from modules.build_dataset import *

In [3]:
#OP_tsv: OpenProt 1.6
#Ens_trx: mart_export (ensembl 95)
#trx_fasta: transcripts fasta (ensembl 95)
#sorfs: list of ribo-seq orfs (mudge & al., 2022)
data = Data(
    OP_tsv = 'data/human-openprot-r1_6-refprots+altprots+isoforms-+uniprot2019_03_01.tsv', 
    Ens_trx = 'data/mart_export.txt',
    trx_fasta = 'data/ensembl95_trxps.fasta',
    sorfs = 'data/sorfs.xlsx'
)

In [4]:
#ensembl_trx = data.ensembl_trx()
#pickle.dump(ensembl_trx, open('data/ensembl_trx.pkl', 'wb'))
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))

In [11]:
#trx_orfs = data.trx_orfs(ensembl_trx)
#pickle.dump(trx_orfs, open('data/trx_orfs.pkl', 'wb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))

In [4]:
#dataset = data.dataset(ensembl_trx, trx_orfs)
#pickle.dump(dataset, open('data/dataset.pkl', 'wb'))
dataset = pickle.load(open('data/dataset.pkl', 'rb'))

In [8]:
#sorfs_dataset = data.sorfs_dataset(ensembl_trx)
#pickle.dump(sorfs_dataset, open('data/sorfs_dataset.pkl', 'wb'))

In [12]:
alt_dataset = data.alt_dataset(ensembl_trx, trx_orfs, alt_orfs)
#pickle.dump(alt_dataset, open('data/alt_dataset.pkl', 'wb'))

In [15]:
pickle.dump(alt_dataset, open('data/alt_dataset.pkl', 'wb'))

In [9]:
alt_orfs = alt_orfs(ensembl_trx)

In [8]:
def alt_orfs(ensembl_trx):
    alt_orfs = []
    with open('data/ncrna_tsv.tsv', 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for n, row in enumerate(reader):
            if n == 0:
                continue
            if n == 1:
                cols = row
                continue
            line = dict(zip(cols, row))
            if 'ENST' not in line["transcript accession"]:
                continue
            if not any(x in line["protein accession"] for x in ["IP_", 'ENSP', "II_"]):
                continue
            trx = line["transcript accession"].split(".")[0]
            altprot = line["protein accession"].split(".")[0]
            alt_orfs.append((trx, altprot))
    return alt_orfs

In [11]:
data.split_dataset(dataset)